Bye


In [59]:
#import imutils
import time
import math
import cv2
import numpy as np
import queue 
from matplotlib.pyplot import bar

previousFrame = None
count = 0
test = 0
temp_frame = None
rect = None
newThresh = None
#return only  the index of the largest contour
def getLargestCountour(contours):
    largestArea = 0
    index = -1
    for i in range(len(contours)):
        area = cv2.contourArea(contours[i])
        if (area>largestArea):
                largestArea=area
                index = i
    if index != -1:
        if cv2.contourArea(contours[index]) < 5000:
            index =-1
    return index       

def searchForMovement(contours, frame):
    #get the index of the largest contour
    index = getLargestCountour(contours)
    
    if index >=0 and index <= len(contours) -1:
        cnt = contours[index]
        M = cv2.moments(cnt) #get some properties from the biggest countour
        cx = int(M['m10']/M['m00']) #x axis centroid
        cy = int(M['m01']/M['m00']) #y axis centroid
        #cv2.circle(frame,(cx,cy),5,[255,0,0],-1) #draw the center of the countour mass
    
        hull = cv2.convexHull(cnt,returnPoints = False)
        minDist = 1000
        frame2 = frame.copy()
        for i in range(len(hull)):
            hullX = cnt[hull[i]][0][0][0]
            hullY = cnt[hull[i]][0][0][1]
            dist = math.sqrt( (hullX - cx)**2 + (hullY - cy)**2 )
            if dist < minDist:
                minDist = dist
            #cv2.circle(frame,(hullX,hullY),5,[255,255,0],-1)
        
        
        cv2.circle(frame,(cx,cy),int(minDist),[255,255,255],0)
        
        circlemask = np.zeros((frame2.shape[0],frame.shape[1]),dtype=np.uint8)
        cv2.circle(circlemask,(cx,cy),int(minDist - 2),(255,255,255),-1,8,0)
        #cv2.imwrite(argv[2],mask)
        out = frame.copy()
        out = cv2.bitwise_and(frame, out, mask=circlemask)
        cv2.imshow("out",out)
        
        #treshhold = 4
        levelRange = 4
        framehsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        circleHueHist = cv2.calcHist([framehsv], [0], circlemask, [256], [0,256])
        treshold1,treshold2 = getHistPeak(circleHueHist,levelRange)   
        #index += int(levelRange / 2)
        lowHue =  treshold1
        highHue = treshold2
        
        '''
        circleSatHist = cv2.calcHist([framehsv], [1], circlemask, [256], [0,256])
        treshold1,treshold2 = getHistPeak(circleSatHist,levelRange)     
        #index += int(levelRange / 2)
        lowSat = treshold1
        highSat = treshold2
        '''
        
        equ = cv2.equalizeHist(framehsv[:,:,1])
        ret2,th2 = cv2.threshold(equ,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        lowSat2, highSat2 = getThreshold(cv2.calcHist([equ],[0],None,[256],[0,256]),ret2)
        #lowSat2 = ret2 - 8
        #highSat2 = ret2 + 8
        '''
        blur = cv2.GaussianBlur(equ,(5,5),0)
        ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        lowSat3 = ret3 - 10
        highSat3 = ret3 + 10
        '''
        circleValHist = cv2.calcHist([framehsv], [2], circlemask, [256], [0,256])
        treshold1,treshold2 = getHistPeak(circleValHist,levelRange)  
        #index += int(levelRange / 2)
        lowVal = treshold1
        highVal = treshold2
        
        #masks1 = mask(framehsv,lowHue,lowSat,lowVal,highHue,highSat,highVal)
        masks2 = mask(framehsv,lowHue,lowSat2,lowVal,highHue,highSat2,highVal)
        
        #asks3 = mask(framehsv,lowHue,lowSat3,lowVal,highHue,highSat3,highVal)
        #cv2.imshow("masks1",masks1)
        cv2.imshow("masks2",masks2)
        #v2.imshow("masks3",masks3)
        #cv2.imshow("mask1s",framehsv[:,:,1])
        '''
        epsilon = 0.01*cv2.arcLength(cnt,True)
        approx = cv2.approxPolyDP(cnt,epsilon,True)
        
        cv2.drawContours(frame, [approx], -1, (0, 0, 255), 3)
        '''
        #cv2.drawContours(frame, hull, -1, (0, 0, 255), 3)
        
        defects = cv2.convexityDefects(cnt,hull)
        #hand = hsv[int(y/3):int(2*y/3),int(x/3):int(2*x/3)]
        #https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_contours/py_contour_features/py_contour_features.html#contour-features
        if defects is not None:
            for i in range(defects.shape[0]):
                s,e,f,d = defects[i,0]
                start = tuple(cnt[s][0])
                end = tuple(cnt[e][0])
                far = tuple(cnt[f][0])
                cv2.line(frame,start,end,[0,255,0],2)
                rows,cols = frame.shape[:2]
                [vx,vy,x,y] = cv2.fitLine(cnt, cv2.DIST_L2,0,0.01,0.01)
                lefty = int((-x*vy/vx) + y)
                righty = int(((cols-x)*vy/vx)+y)
                cv2.line(frame,(cols-1,righty),(0,lefty),(0,0,255),2)

    return frame

def trackMotion(frame,prev, sensitivity_value):

        # Convert to grayscale and blur it for better frame difference
        # frame = cv2.bilateralFilter(frame, 7, 150, 150)
        frame = cv2.blur(frame, (9,9)) # blur the original frame "very important for the final image"


        previousFrame = prev
        global newThresh

        
        kernelErosion = np.ones((1,1),np.uint8) #kernel for erosion
        kernelDilation = np.ones((8,8),np.uint8) #kernel for dilation
        if previousFrame is None:
            previousFrame = frame.copy()
            prevHist =0
            #print('hello')
            return frame, previousFrame, frame, frame   

        frameDiff = cv2.absdiff(previousFrame, frame)
        #frameDiff = cv2.cvtColor(frameDiff, cv2.COLOR_BGR2GRAY)
        
        thresh = cv2.threshold(frameDiff, sensitivity_value, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
      
        hist = cv2.calcHist([thresh], [0], None, [256], [0,256])

        if hist[255] >= 5000 :
            previousFrame = frame.copy()
            newThresh = thresh # important
            erosion = cv2.erode(newThresh,kernelErosion,iterations =1)
            newThresh = cv2.dilate(newThresh, kernelDilation, iterations=4)
            newThresh = cv2.threshold(newThresh, 250, 255, cv2.THRESH_BINARY)[1]
    

        if newThresh is None:
                newThresh = thresh
        
        #find the contours from the remaining threshold image 
        _, cnts, _ = cv2.findContours(newThresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        #search for any movement
        frame = searchForMovement(cnts, frame)    
        
        #show images
        cv2.namedWindow('frame',cv2.WINDOW_NORMAL)
        cv2.resizeWindow('frame', 800,600)
        cv2.imshow("frame", frame)
        cv2.namedWindow('newThresh',cv2.WINDOW_NORMAL)
        cv2.resizeWindow('newThresh', 800,600)
        cv2.imshow("newThresh", newThresh)
        
        
        return frame, previousFrame, thresh, frameDiff


if __name__ == '__main__':


    camera =cv2.VideoCapture("output2.avi")  # this for mobil connection cam
    #camera =cv2.VideoCapture(0)
    time.sleep(0.25)
    #global hists
    #hists = queue.Queue(10)
    #global prevSum
    #prevSum = 0
    global previousFrame
    #previousFrame = None
    while camera.isOpened():
        
        sensitivity_value = 20
        ret, frame = camera.read()

        #Check if the next camera read is not null
        if ret:
            frame, previousFrame, thresh, frameDiff = trackMotion(frame,previousFrame, sensitivity_value)


        else:
            print("Video Finished")
            close = False
            while not close:
                key1 = cv2.waitKey(3) & 0xFF
                if key1 == 27 or key1 == ord('q'):
                    close = True

            break
        key = cv2.waitKey(3) & 0xFF
        if key == 27 or key == ord('q'):
            print("Bye")
            break

    camera.release()
    cv2.destroyAllWindows()

5 126.0 68
10 126.0 95
105 121.0 56
110 122.0 103
101 121.0 33
110 121.0 106
104 125.0 62
109 126.0 118
5 124.0 64
12 124.0 74
104 124.0 48
110 124.0 118
104 124.0 51
110 124.0 115
103 126.0 61
110 126.0 92
101 124.0 63
110 125.0 87
104 121.0 67
108 122.0 90
103 123.0 67
108 124.0 86
103 122.0 69
108 123.0 89
103 121.0 68
109 122.0 99
9 125.0 68
14 125.0 88
10 125.0 68
14 125.0 89
9 125.0 69
14 125.0 92
10 123.0 70
15 124.0 91
9 123.0 71
14 125.0 86
10 126.0 70
14 127.0 85
10 124.0 71
14 126.0 86
10 127.0 70
15 130.0 85
9 126.0 71
14 128.0 86
9 127.0 70
14 129.0 87
9 125.0 71
15 127.0 83
12 126.0 68
16 128.0 85
12 128.0 69
15 130.0 86
12 127.0 69
16 128.0 86
11 128.0 69
15 130.0 86
11 127.0 69
16 128.0 86
11 127.0 68
15 128.0 86
11 129.0 69
15 129.0 86
11 129.0 69
16 130.0 85
11 127.0 69
16 128.0 84
11 128.0 68
15 130.0 85
12 126.0 70
15 128.0 86
11 129.0 70
15 130.0 86
12 123.0 70
15 128.0 86
11 129.0 70
15 132.0 86
12 127.0 70
15 129.0 86
12 129.0 70
15 131.0 86
12 126.0 70
15 128.0 

In [24]:
def getHistPeak(Hist,levelRange):
    PixelsCount = 0
    maxIndex = 0
    maxPixels = 0
    for i in range(256-levelRange):
        for j in range(i,i+levelRange):
            PixelsCount += Hist[j]
        if PixelsCount > maxPixels:
            maxIndex = i
            maxPixels = PixelsCount
        PixelsCount = 0
    startIndex , endIndex = getThreshold(Hist,maxIndex + int(levelRange/2))
    return startIndex , endIndex

In [34]:
def getThreshold(hist,maxIndex):
    threshold1 = maxIndex - 1
    threshold2 = maxIndex + 1
    stop = False
    while threshold1 >= 0 and not(stop):
        if hist[int(threshold1)] >= (0.4 * hist[int(maxIndex)]):
            threshold1 = threshold1 - 1
        else:
            stop = True
    stop = False
    while threshold2 < 256 and not(stop):
        if hist[int(threshold2)] >= (0.4 * hist[int(maxIndex)]):
            threshold2 = threshold2 + 1
        else:
            stop = True
    return threshold1 + 1, threshold2 - 1

In [58]:
def mask(frame,lowHue,lowSat,lowVal,highHue,highSat,highVal):
   #this is the limit threshold the lowest from the average value and the highest from the average value
    try:
        print(lowHue,lowSat,lowVal)
        print(highHue,highSat ,highVal)
        
        kernelDilation = np.ones((2,2),np.uint8) #kernel for dilation
        lower_blue = np.array([lowHue  ,lowSat - 255 ,lowVal])
        upper_blue = np.array([highHue ,highSat + 255,highVal ])
                
        mask0 = cv2.inRange(frame, lower_blue, upper_blue)
        
        lower_blue = np.array([lowHue  ,lowSat - 255 ,lowVal -255])
        upper_blue = np.array([highHue ,highSat + 255,highVal + 255 ])
        
        mask1 = cv2.inRange(frame, lower_blue, upper_blue)
        
        mask1 = cv2.dilate(mask1,kernelDilation,iterations = 1)
        
        lower_blue = np.array([lowHue - 255  ,lowSat , lowVal -255])
        upper_blue = np.array([highHue + 255 ,highSat ,highVal + 255 ])
        
        mask2 = cv2.inRange(frame, lower_blue, upper_blue)
        
        lower_blue = np.array([lowHue - 255  ,lowSat -255  ,lowVal])
        upper_blue = np.array([highHue + 255 ,highSat + 255 ,highVal])
        
        mask3 = cv2.inRange(frame, lower_blue, upper_blue)
        
        mask = mask0
        #mask = cv2.dilate(mask0,kernelDilation,iterations = 2)

        
    except Exception as e:
        print(e)
        return frame
    
    

    return mask